<a href="https://colab.research.google.com/github/suyash091/EEG-MULTIPLE-CHANNEL/blob/master/14%20channel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!git clone https://github.com/tfjgeorge/ml-project.git

Cloning into 'ml-project'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 141 (delta 54), reused 141 (delta 54), pack-reused 0
Receiving objects: 100% (141/141), 4.42 MiB | 7.28 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [0]:
!mv '/content/ml-project' '/content/drive/My Drive'

In [0]:
from os.path import isfile
import zipfile
import numpy as np
import random

dataset_path = './data/EP.zip'

def get_dataset_file():
    if not isfile(dataset_path):
        import urllib
        origin = (
            'http://mindbigdata.com/opendb/MindBigData-EP-v1.0.zip'
        )
        print('Downloading data from %s' % origin)
        urlretrieve(origin, dataset_path)
    return open(dataset_path, 'rb')

def get_datasets():
    f = get_dataset_file()
    zf = zipfile.ZipFile(f)
        
    data = [ line for line in zf.open('EP1.01.txt')]
    entire_dataset = []
    current_event = np.zeros(256 * 14 + 2)
    
    print('Reading data file')
    i = 0

    for l in data:
        #print(str(l))
        ids, event, device, channel, code, size, data = l.decode("utf-8").split('\t')

        signals = np.array([float(val) for val in data.split(',')])
        
        current_event[1+ i*256:1+ i*256 + min(len(signals), 256)] = signals[:256]
        i += 1

        if i == 14: # we assume all channels from an event are in sequence
            current_event[-1] = int(code)
            current_event[0] = min(len(signals), 256)

            entire_dataset.append(current_event)
            current_event = np.zeros(256 * 14 + 2)
            i = 0

    random.seed(111) # deterministic
    random.shuffle(entire_dataset)

    entire_dataset = np.array(entire_dataset)
    return entire_dataset[:60000], entire_dataset[60000:]

def split_into_subsequences(data, n_sequences, length):
    output = np.zeros((data.shape[0]*n_sequences, length*14+1))
    for i in range(data.shape[0]):
        if n_sequences == 1:
            steps = 0
        else:
            steps = (data[i, 0] - length) / (n_sequences - 1)

        for j in range(n_sequences):
            output[i*n_sequences+j,:length]           = data[i, j*steps: j*steps+length]
            output[i*n_sequences+j,length:length*2]   = data[i, j*steps+512: j*steps+512+length]
            output[i*n_sequences+j,length*2:length*3] = data[i, j*steps+512*2: j*steps+512*2+length]
            output[i*n_sequences+j,length*3:length*4] = data[i, j*steps+512*3: j*steps+512*3+length]
            output[i*n_sequences+j,-1] = data[i, -1]

    return output

In [0]:

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from urllib.request import urlretrieve
#from sklearn.gaussian_process import GaussianProcess

In [76]:
(25/2050)*3586

43.73170731707317

In [3]:
train, test = get_datasets()
print(train.shape)
print(test.shape)

Reading data file
(60000, 3586)
(5034, 3586)


In [0]:
#train = train[:int(len(train)/3),:]
#test = test[:int(len(test)/3), :]

#train[train[:,-1] >= 0,-1] =  0
#test[test[:,-1] >= 0,-1] =  0

#train[:,-1] = train[:,-1] + 1
#test[:,-1] = test[:,-1] + 1

train_target = train[:,-1]
test_target = test[:,-1]

DECISIONBOUNDARY = 0

In [0]:
#FastFourierTransformation apply to the sets using the 4 sensors

train_f = np.zeros(train.shape)

for i in range(len(train_f)):
    length = train[i][0]
    train_f[i][0] = train[i][0]
    train_f[i][-1] = train[i][-1]
    
    for j in range(14):
        train_f[int(i)][int(1+256*j):int(1+256*j+length)] = np.abs(np.fft.fft(train[int(i)][int(1+256*j):int(1+256*j+length)]))
        
test_f = np.zeros(test.shape)

for i in range(len(test_f)):
    length = test[i,0]
    test_f[i,0] = test[i,0]
    test_f[i,-1] = test[i,-1]
    
    for j in range(14):
        test_f[i][int(1+256*j):int(1+256*j+length)] = np.abs(np.fft.fft(test[i][int(1+256*j):int(1+256*j+length)]))

In [0]:
#PCA with 25 components
pca = PCA(n_components = 45)
train_principal = pca.fit_transform(train_f)
test_principal = pca.transform(test_f)

In [7]:
test_principal.shape

(5034, 25)

In [98]:
#KNN
erreur = []
x = np.arange(1,20,1)
for n in (x):
    # train set
    neigh = KNeighborsClassifier(n_neighbors=n)
    neigh.fit(train_principal, train[:,-1])

    #test set
    total_correct = 0
    for i in range(test_principal.shape[0]):
        if neigh.predict(test_principal[i].reshape(1, -1))[0] == test[i,-1]:
            total_correct += 1

    erreur.append(float(total_correct) / test_principal.shape[0] * 100)
    print('Pourcentage correct %d:' % n, float(total_correct) / test_principal.shape[0] * 100)

Pourcentage correct 1: 10.389352403655144
Pourcentage correct 2: 10.647596344854986
Pourcentage correct 3: 10.707191100516487
Pourcentage correct 4: 10.985299960270163
Pourcentage correct 5: 11.22367898291617
Pourcentage correct 6: 10.985299960270163
Pourcentage correct 7: 11.0647596344855
Pourcentage correct 8: 11.005164878823996
Pourcentage correct 9: 10.687326181962655
Pourcentage correct 10: 10.46881207787048


KeyboardInterrupt: ignored

In [0]:
#Import models from scikit learn module:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
import numpy as np
from sklearn.metrics import roc_curve, auc, precision_score, confusion_matrix, explained_variance_score, max_error, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso

In [104]:

model = RandomForestClassifier(n_estimators=100)
model.fit(train_principal,train[:,-1])


#predictions = model.predict(test_principal)
prc=precision_score(predictions,test[:,-1], average=None)
cfm=confusion_matrix(predictions,test[:,-1])
accuracy = metrics.accuracy_score(predictions,test[:,-1])
print(prc,cfm,accuracy)
#print(explained_variance_score(predictions,test))
#print(max_error(predictions,test))
#print(mean_absolute_error(predictions, test, multioutput='raw_values'))
#print(mean_squared_error(predictions, test, multioutput='raw_values'))

[0.93333333 0.14688129 0.12277228 0.13609467 0.12549801 0.1085595
 0.10931174 0.12783505 0.11018711 0.09760589 0.08365019] [[14  0  0  0  0  0  1  0  0  0  0]
 [ 0 73 60 55 56 62 57 48 54 58 57]
 [ 0 45 62 52 51 51 54 58 36 60 69]
 [ 0 40 53 69 59 42 53 48 58 47 49]
 [ 0 66 59 65 63 59 49 54 57 63 68]
 [ 1 50 52 42 41 52 52 45 38 55 38]
 [ 0 52 50 53 49 47 54 52 53 58 59]
 [ 0 45 44 46 44 46 49 62 55 59 56]
 [ 0 39 34 48 55 39 34 32 53 46 45]
 [ 0 43 46 52 39 39 41 41 44 53 41]
 [ 0 44 45 25 45 42 50 45 33 44 44]] 0.11899086213746524


In [10]:
model = LogisticRegression()
model.fit(train_principal,train[:,-1])


predictions = model.predict(test_principal)`≈`
prc=precision_score(predictions,test[:,-1], average=None)
cfm=confusion_matrix(predictions,test[:,-1])
accuracy = metrics.accuracy_score(predictions,test[:,-1])
print(prc,cfm,accuracy)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[1.         0.09859155 0.12871287 0.         0.03585657 0.10438413
 0.         0.00206186 0.21413721 0.3572744  0.02661597] [[ 15  34  26  34  43  26  36  31  33  37  36]
 [  0  49  62  47  51  47  48  36  41  44  60]
 [  0  73  65  62  60  64  60  73  50  71  66]
 [  0   0   0   0   1   0   0   0   0   0   1]
 [  0  14  14  23  18  19  13   7  20  17   9]
 [  0  54  71  44  45  50  47  59  45  52  54]
 [  0   4   2   3   1   0   0   1   1   1   2]
 [  0   2   0   0   0   1   1   1   0   1   1]
 [  0  89  95  96  88 106  87  97 103 109  94]
 [  0 160 157 182 186 155 192 163 182 194 189]
 [  0  18  13  16   9  11  10  17   6  17  14]] 0.1011124354390147


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [14]:
model = LinearRegression()
#model.fit(train_principal,train[:,-1])


#predictions = model.predict(test_principal)
#prc=precision_score(predictions,test[:,-1], average=None)
print(explained_variance_score(predictions,test[:,-1]))
print(max_error(predictions,test[:,-1]))
print(mean_absolute_error(predictions, test[:,-1], multioutput='raw_values'))
print(mean_squared_error(predictions, test[:,-1], multioutput='raw_values'))

-1547.456880419968
5.408471123009611
[2.53968557]
[8.46931246]


In [16]:
model = RandomForestRegressor()
model.fit(train_principal,train[:,-1])


predictions = model.predict(test_principal)
print(explained_variance_score(predictions,test[:,-1]))
print(max_error(predictions,test[:,-1]))
print(mean_absolute_error(predictions, test[:,-1], multioutput='raw_values'))
print(mean_squared_error(predictions, test[:,-1], multioutput='raw_values'))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


-9.092235523960634
7.6
[2.60101311]
[9.21003774]


In [19]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

!pip install scipy
from scipy import signal

In [21]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1024, input_dim=45, activation='relu'))
model.add(tf.keras.layers.Dense(4096, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(4096, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(4096, activation='relu'))
model.add(tf.keras.layers.Dense(11, activation='softmax'))
alpha = 2e-3
batch_size = 256
num_epochs = 3000
model.compile(
      optimizer=tf.train.GradientDescentOptimizer(learning_rate=alpha, ),
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      metrics=['sparse_categorical_accuracy', 'accuracy']
  )

History = model.fit(
      x = train_principal,
      y = train[:,-1],
      batch_size = batch_size,
      epochs=num_epochs,
      #validation_data = (x_valid, y_valid),
      #callbacks = [checkPointer,tensorBoard]
  )
predictions = model.predict(test_principal)

Train on 60000 samples
Epoch 1/3000
60000/60000 [==============================] - 3s 58us/sample - loss: nan - sparse_categorical_accuracy: 0.1003 - acc: 0.1003
Epoch 2/3000
60000/60000 [==============================] - 3s 44us/sample - loss: nan - sparse_categorical_accuracy: 0.1003 - acc: 0.1003
Epoch 3/3000
60000/60000 [==============================] - 3s 44us/sample - loss: nan - sparse_categorical_accuracy: 0.1003 - acc: 0.1003
Epoch 4/3000
60000/60000 [==============================] - 3s 44us/sample - loss: nan - sparse_categorical_accuracy: 0.1003 - acc: 0.1003
Epoch 5/3000
60000/60000 [==============================] - 3s 44us/sample - loss: nan - sparse_categorical_accuracy: 0.1003 - acc: 0.1003
Epoch 6/3000
60000/60000 [==============================] - 3s 44us/sample - loss: nan - sparse_categorical_accuracy: 0.1003 - acc: 0.1003
Epoch 7/3000
60000/60000 [==============================] - 3s 44us/sample - loss: nan - sparse_categorical_accuracy: 0.1003 - acc: 0.1003
Epoc

KeyboardInterrupt: ignored